In [14]:
import os
from pymongo import MongoClient
import certifi
from dotenv import load_dotenv
import pandas as pd

In [9]:
load_dotenv("../.env.prod")

True

In [10]:
mongo_client = MongoClient(os.getenv("MONGODB_URI"), tlsCAFile=certifi.where())
db = mongo_client["harvester"]
payloads_collection = db["payloads"]

In [18]:
#: get all payloads, grouped by thx_guid, and sorted by timestamp
pipeline = [
    {"$group": {"_id": "$thx_guid", "payloads": {"$push": "$$ROOT"}}},
    {"$sort": {"payloads.timestamp": 1}},
]
cursor = payloads_collection.aggregate(pipeline)

In [19]:
#: convert to dataframe
df = pd.DataFrame.from_records(cursor)

In [20]:
df.count()  #: amount of sessions

_id         286
payloads    286
dtype: int64

In [23]:
df.to_csv("sessions.csv", index=False)